In [ ]:
# First we import the module and start up the Matlab interpreter

import os
import numpy as np
from pace_neutrons import Matlab
m = Matlab()

# Note that every Matlab command (including all Horace and SpinW commands)
# must be called as a method of the Matlab `m` object.
# For example, to calculate the eigenvalues of a 3x3 "magic" square matrix in Matlab:
em = m.eig(m.magic(3))
print(em)

# Also we switch on the parallel processing capabilities of Horace
m.hpc('on')
m.hpc_config().parallel_cluster = 'herbert'
m.hpc_config().combine_sqw_using = 'mpi_code'
m.hpc_config().build_sqw_in_parallel = True

----

We now run a GUI tool call `horace_planner` which can be used to plan an experiment

You can get help on it from here:

https://pace-neutrons.github.io/Horace/unstable/manual/Planning_a_Horace_scan.html

In [ ]:
%matlab_plot_mode windowed
m.horace_planner()


----

As an alternative, you can use the `dummy_sqw` command to generate a full SQW dataset and use the normal Horace plot and cut commands to view what your data would look like.

Help can be obtained here:

https://pace-neutrons.github.io/Horace/unstable/manual/Planning_a_Horace_scan.html#more-detailed-planning-of-scans

In [ ]:
output_data_folder = '/tmp/'  # Or you home folder
instrument_folder = '/mnt/ceph-training/course_materials/horace_instrument_files'

# Name and folder for output "fake" generated file
sqw_file = os.path.join(output_data_folder, 'my_dummy_file.sqw')

# Instrument parameter file (may be in another location to this)
par_file = os.path.join(instrument_folder, 'MAPS_4to1_102.par')

# In this case we want to simulate a measurement of bcc-Iron, which has lattice
# parameter 2.87 Angstrom, and the crystal was mounted in the H-K scattering plane
# We will be using an incident energy of 400 meV

# u and v vectors to define the crystal orientation 
# (u||ki, uv plane is horizontal but v does not need to be perp to u.
# See: https://pace-neutrons.github.io/Horace/unstable/user_guide/Getting_started.html#example
# For an image of the coordinate system used by Horace
u = [?, ?, ?]
v = [?, ?, ?]

# Range of rotation (psi) angles to cover in simulated dataset.
# (psi=0 when u||ki)
psi = range(?, ?, ?)

# Incident energy in meV
efix = ?
emode = 1    # This is for direct geometry (set to 2 for indirect)

# Range of energy transfer (in meV) for the dataset to cover
en = range(?, ?, ?)

# Sample lattice parameters (in Angstrom) and angles (in degrees)
alatt = ?
angdeg = ?

# Sample misalignment angles ("gonios"). [More details in session 4].
omega=0; dpsi=0; gl=0; gs=0;

# This runs the command to generate the "fake" dataset.
m.dummy_sqw (en, par_file, sqw_file, efix, emode, alatt, angdeg,
            u, v, psi, omega, dpsi, gl, gs)

In [ ]:
# =========================================================================
# Once generated, you can use standard Horace plotting tools to explore 
# this fake dataset, where the colour scale corresponds to the value of psi
# that contributed data to a given region of reciprocal space					 

sqw_file = os.path.join(output_data_folder, 'my_dummy_file.sqw')

# First define a view projection specified by two vectors, u and v.
# (Note that these u and v do not need to be the same as the sample u and v above.
# They just define the first, second and third axes for making a cut 
# (third axis w is implicit being perpendicular to the plane defined by u and v).
u = [?, ?, ?]
v = [?, ?, ?]

# We also define a coordinate offset (the 4th offset coordinate is energy transfer)
offset = [0, 0, 0, 0]

# We also define the Q (momentum) units for each axis which can be either
# 'r' for r.l.u. or 'a' for absolute (A^-1). This is defined for each u, v, w direction.
# E.g. 'rar' means u and w are normalissed to in r.l.u, and v in A^-1.
projtype = 'rrr'

proj = m.line_proj(u, v, 'offset', offset, 'type', projtype)

# You can also use the more Pythonic:
#proj = m.line_proj(u, v, offset=offset, type=projtype)

# Now make a cut of the fake dataset.
# The four vectors indicate either the range and step (three-vector) or
# the integration range (2-vector), with units defined by the proj.type
# The following makes a 3D volume cut with axes u, v and energy 
# (first, second and fourth vectors are 3-vectors), 
# integrating over w between -0.1 and 0.1.
# '-nopix' indicates to discard the pixel information and create
# a dnd (d3d) object.
my_vol = m.cut_sqw(sqw_file, proj, [?,?,?], [?,?,?],
                   [-0.1, 0.1], [?,?,?], '-nopix')

%matlab_plot_mode windowed

# Plot the 3D cut - click on the graph to plot 2D projections of the volume
my_vol.plot()
#m.uiwait()

In [ ]:
%matlab_plot_mode inline

# The following makes a 2D cut with axes u and energy (first and fourth
# vectors are 3-vectors), integrating over v and w between -0.1 and 0.1
my_cut = m.cut_sqw(sqw_file, proj, [?,?,?], [-0.1,0.1], [-0.1,0.1], [?,?,?], '-nopix')

# Now plot the 2D cut.
m.plot(my_cut)
m.keep_figure()

# Now make a set of three figures, plotting the dispersion along the [H00] direction
# but centered on (200), (020), and (002) in turn. Use keep_figure after each figure.
proj = m.line_proj([?,?,?], [?,?,?], 'offset', [?,?,?,?], 'type', 'rrr')
m.plot(m.cut_sqw(sqw_file, proj, [?,?,?], [-0.1,0.1], [-0.1,0.1], [0,4,360], '-nopix'))
m.keep_figure();

proj = m.line_proj([?,?,?], [?,?,?], 'offset', [?,?,?,?], 'type', 'rrr')
m.plot(m.cut_sqw(sqw_file, proj, [?,?,?], [-0.1,0.1], [-0.1,0.1], [0,4,360], '-nopix'))
m.keep_figure();

proj = m.line_proj([?,?,?], [?,?,?], 'offset', [?,?,?,?], 'type', 'rrr')
m.plot(m.cut_sqw(sqw_file, proj, [?,?,?], [-0.1,0.1], [-0.1,0.1], [0,4,360], '-nopix'))
m.keep_figure();

# Now make a 2D slice along [HHH] centred at (200)
proj = m.line_proj(?)
m.plot(m.cut_sqw(sqw_file, proj, ?, ?, ?, ?, '-nopix'))

----

In the next part we will create an SQW dataset from a crystal rotation measurement of bcc-Iron on MAPS.

The lattice parameter is 2.87 Angstrom, and the crystal was mounted in the [HK0] scattering plane.

`Ei=400` was used, and there are 46 datafiles covering the range 0 to 90 degrees in scattering angle (with 0 being the [H00] direction) in 2 degree steps.

In [ ]:
output_data_folder = '/tmp/'  # Or you home folder
data_path = '/mnt/ceph-training/course_materials/horace_datafiles'

# Name of output sqw file (for the 4D combined dataset)
sqw_file = os.path.join(output_data_folder, 'iron.sqw')

# Instrument parameter file name (only needed for spe files - nxspe files
# have the par file embedded in them).
par_file = ''

# u and v vectors to define the crystal orientation 
# (u||ki when psi=0; uv plane is horizontal but v does not need to be perp to u).
u = [?, ?, ?] 
v = [?, ?, ?]

# Range of rotation (psi) angles of the data files.
# (psi=0 when u||ki)
psi = range(?, ?, ?)

# Data file run number corresponding to the psi angles declared above
# (must be the same size and order as psi)
runno = range(15052, 15098)

# Ensure that we have the parallised SQW generation switched on
m.hpc('on')
m.hpc_config().parallel_cluster = 'herbert'
m.hpc_config().combine_sqw_using = 'mpi_code'
m.hpc_config().build_sqw_in_parallel = True

# Incident energy in meV
efix = ?
emode = 1   # This is for direct geometry (set to 2 for indirect)

# Sample lattice parameters (in Angstrom) and angles (in degrees)
alatt = [?, ?, ?];
angdeg = [?, ?, ?];

# Sample misalignment angles ("gonios"). [More details in session 4].
omega=0; dpsi=0; gl=0; gs=0;

# Construct the data file names from the run numbers (the data file names
# are actually what is required by the gen_sqw function below, but we
# use the numbers as a convenience. This assumes that the data file names
# follow the standard convention of IIInnnnnn_eiEEE.nxspe, where III is
# the instrument abbreviation (MAP, MER or LET), nnnnnn is the run number
# and EEE is the incident energy.
efix_for_name = 400;
spefile = [f'{data_path}/map{runno[ii]}_ei{efix_for_name}.nxspe' for ii in range(len(psi))]

# Now run the function to generate the sqw file. This will take ~5 min
m.gen_sqw(spefile, par_file, sqw_file, efix, emode, alatt, angdeg, u, v, psi, omega, dpsi, gl, gs)


Now make cuts of the data. E.g.

* A 3D plot with Q axes along [HH0] and [-H,H,0] and E. 
* A 2D Q-E cut along the [HH0] direction
* A set of 1D cuts along [HH0] integrating over different energy transfer ranges from [50,70], [70,90], [90,110], [110,130] meV. 

In [ ]:
%matlab_plot_mode windowed

# Now make a cut and plot it
proj = m.line_proj([?,?,?], [?,?,?], 'type', 'rrr')
w1 = m.cut_sqw(sqw_file, proj, ?, ?, ?, ?);
hf = w1.plot()
#m.uiwait(hf)

### Accumulating data to an existing sqw file

The above `gen_sqw` command generates an sqw file from the list of input `spe` or `nxspe` files in one go, and deletes all temporary files after it finishes. If you are in the middle of a rotation scan, you can use `accumulate_sqw` which does not delete the temporary files and so can append newly processed `spe`/`nxspe` files to an existing sqw file.

This may save some time in processing, but is not now generally recommended since the implementation of parallelisation in `gen_sqw` has made `gen_sqw` much faster. This is because `accumulate_sqw` needs to know _all_ the *psi* values (including those not yet measured) in order to construct the coarse data grid that enables Horace to make fast cuts. If you then include measurements at *psi* values not in the original list, then it is possible that some data will lie outside this grid and it will be 'lost' to the sqw file. If the additional runs are ones that interleave between the original files, this will not be a problem, but if the additional runs extend the original angular range, then you must use the 'clean' option which is equivalent to running `gen_sqw`.

In addition, if any of the temporary `tmp` corresponding to each `spe`/`nxspe` file are corrupted then `accumulate_sqw` will fail and you must then either delete the corrupted `tmp` file or use the 'clean' option.

The syntax for accumulate_sqw is very similar to gen_sqw:

```
m.accumulate_sqw(spefile, par_file, sqw_file, efix, emode, alatt, angdeg,
                 u, v, psi, omega, dpsi, gl, gs)
```

Or:

```
m.accumulate_sqw(spefile, par_file, sqw_file, efix, emode, alatt, angdeg,
                  u, v, psi, omega, dpsi, gl, gs, 'clean')
```

This is a way of appending newly processed spe files to an existing dataset. The key point is that the `psi` and `spe_file` arrays contain a list of *planned* files and run-numbers - only those that actually exist will be included in the file.

You can run this periodically, for example overnight.

In [ ]:
# =========================================================================
#                     Basic customisation of plots
# =========================================================================

# Make axes tight:
m.plot(m.compact(my_slice));

# Smoothing:
#m.plot(m.smooth(my_slice))   # this gives an error - think about why!

m.plot(m.smooth(m.d2d(my_slice)));
# d2d command (for 2d objects) converts from sqw type data, with detector pixel retained
# to d2d / dnd object that is smaller in memory and without detector pixel info

# Smoothing options:
m.plot(m.smooth(m.d2d(my_slice),[2,2],'gaussian'));

# Set colour scale and other axes scales in script:
m.lz(0, 0.5)
m.ly(50, 250)
m.lx(-1.5, 1.5)

# Reset a limit
m.lx()

# Retain a figure, so it is not replaced next time you make a plot (of the
# same dimensionality)
m.keep_figure()
m.plot(my_slice)

# Cursor to find a particular data point value
m.plot(my_cut2)
try:
    m.xycursor()
except:
    pass
